In [ ]:
# Vereiste imports

from sklearn.metrics import classification_report
from keras import applications, optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.initializers import RandomNormal
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os, keras, cv2

In [ ]:
# Zorg voor reproduceerbaarheid door gebruik te maken van een seed-waarde.

keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=42)

# Leg de afmetingen van de input-afbeeldingen vast.

img_width, img_height = 224, 224

# Locatie dataset.

train_data_dir = '/path/to/data/here'

# Leg het aantal epochs en de batch size vast.

epochs_run1 = 10
epochs_run2 = 10
batch_size = 16

# Voorbereidingen voor data augmentation, genereren van train- en test-afbeeldingen

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=5,
    zoom_range=0,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# Implementeer early stopping. 

early_stop = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=4, mode='auto', verbose=1)

In [ ]:
# RUN 1 - Train alleen het eigen deel van het netwerk.

# Importeer VGG16, het neurale netwerk dat we gebruiken voor transfer learning.

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))
print('Base model loaded.')

# Voeg het 'eigen' deel van het netwerk toe.

x = Flatten(name='flatten')(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x) # Aanpasbaar naar aantal categorieen.
model = Model(inputs= base_model.input, outputs=x)
print('This is the number of trainable weights '
      'before freezing part of the network:', len(model.trainable_weights))

# Bevries de eerste 20 lagen van het netwerk.

for layer in model.layers[:20]:
    layer.trainable = False
    print('This is the number of trainable weights '
      'after freezing part of the network:', len(model.trainable_weights))

# Print overzicht van het netwerk.

model.summary()

# Compile het model, learning rate kan aangepast worden.

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# Train het netwerk.

model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=epochs_run1,
                    validation_data=validation_generator,
                    validation_steps= validation_generator.samples // batch_size,
                    verbose=1, callbacks = [early_stop])

# Sla de resultaten op.

model.save('run1.h5')
model.save_weights('run1-weights.h5')
print('Run 1 saved.')

In [ ]:
# RUN 2 - Train eigen deel van het netwerk en het laatste deel van het geimporteerde netwerk.

# Initialiseer een checkpoint voor het opslaan van de gewichten van het netwerk.

checkpoint = ModelCheckpoint('run2_best_weights.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto', period=1)

# Importeer het model uit run 1.

model_copy = keras.models.load_model('run1.h5')

# Maak het laatste deel van het geimporteerde netwerk trainable.

for layer in model_copy.layers:
    layer.trainable = True
for layer in model_copy.layers[:15]:
    layer.trainable = False
model_copy.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
model_copy.load_weights('run1-weights.h5')
print('This is the number of trainable weights '
      'after unfreezing part of the network:', len(model_copy.trainable_weights))

# Print overzicht van het netwerk.

model_copy.summary()

# Train het netwerk.

M = model_copy.fit_generator(train_generator,
                        steps_per_epoch=train_generator.samples // batch_size,
                        epochs=epochs_run2,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.samples // batch_size,
                        verbose=1, callbacks = [early_stop,checkpoint])

In [ ]:
#Leg het verloop van de accuracy en loss vast

acc = M.history['acc']
val_acc = M.history['val_acc']
loss = M.history['loss']
val_loss = M.history['val_loss']
num_epochs = range(len(acc))

# Visualizeer verloop accuracy

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epochs_run2), M.history["acc"], label="train_acc")
plt.plot(np.arange(0, epochs_run2), M.history["val_acc"], label="val_acc")
plt.title("Training Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="upper left")
plt.show()

plt.plot(num_epochs, acc, 'bo', label='Training acc')
plt.plot(num_epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

# Visualizeer verloop loss

plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epochs_run2), M.history["loss"], label="train_loss")
plt.plot(np.arange(0, epochs_run2), M.history["val_loss"], label="val_loss")
plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.show()

plt.plot(num_epochs, loss, 'bo', label='Training loss')
plt.plot(num_epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# Visualizeer de activaties uit het laatste deel van het netwerk

from keras import backend as K

def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def generate_pattern(layer_name, filter_index, size=150):
    # Build a loss function that maximizes the activation
    # of the nth filter of the layer considered.
    layer_output = model_copy.get_layer(layer_name).output
    loss = K.mean(layer_output[:, :, :, filter_index])

    # Compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, model_copy.input)[0]

    # Normalization trick: we normalize the gradient
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

    # This function returns the loss and grads given the input picture
    iterate = K.function([model_copy.input], [loss, grads])
    
    # We start from a gray image with some noise
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.

    # Run gradient ascent for 40 steps
    step = 1.
    for i in range(40):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
        
    img = input_img_data[0]
    return deprocess_image(img)

plt.imshow(generate_pattern('block5_conv1', 0))
plt.grid(False)
plt.show()

In [ ]:
# Laat het netwerk een voorspelling doen op basis van een aangeleverde image
# en visualiseer de resultaten

# Locatie van de input image

img_path = '/path/to/image.jpg'

# Locatie waar het gevisualiseerde resultaat kan worden opgeslagen

output_path = '/path/to/output.jpg'

# `img` is a PIL image of size 224x224
img = image.load_img(img_path, target_size=(224, 224))

# `x` is a float32 Numpy array of shape (224, 224, 3)
x = image.img_to_array(img)

# We add a dimension to transform our array into a "batch" of size (1, 224, 224, 3)
x = np.expand_dims(x, axis=0)

# Finally we preprocess the batch (this does channel-wise color normalization)
x = preprocess_input(x)

keys=list(validation_generator.class_indices.keys())
y_prob = model_copy.predict(x) 
y_classes = y_prob.argmax(axis=-1)
print('Predicted category:', (keys[(np.asscalar(y_classes))]))

# This is the entry in the prediction vector
img_output = model_copy.output[:, (np.asscalar(y_classes))]

# The is the output feature map of the `block5_conv3` layer, the last convolutional layer in VGG16
last_conv_layer = model_copy.get_layer('block5_conv3')

# This is the gradient of the predicted class with regard to the output feature map of `block5_conv3`
grads = K.gradients(img_output, last_conv_layer.output)[0]

# This is a vector of shape (512,), where each entry is the mean intensity of the gradient over a specific feature map channel
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# This function allows us to access the values of the quantities we just defined:
# `pooled_grads` and the output feature map of `block5_conv3`,
# given a sample image
iterate = K.function([model_copy.input], [pooled_grads, last_conv_layer.output[0]])

# These are the values of these two quantities, as Numpy arrays,
# given our sample image of two elephants
pooled_grads_value, conv_layer_output_value = iterate([x])

# We multiply each channel in the feature map array
# by "how important this channel is" with regard to the elephant class
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# The channel-wise mean of the resulting feature map
# is our heatmap of class activation
heatmap = np.mean(conv_layer_output_value, axis=-1)

# Plot heatmap
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

# We use cv2 to load the original image
img = cv2.imread(img_path)

# We resize the heatmap to have the same size as the original image
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

# We convert the heatmap to RGB
heatmap = np.uint8(255 * heatmap)

# We apply the heatmap to the original image
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# 0.4 here is a heatmap intensity factor
superimposed_img = heatmap * 0.4 + img

# Plot image/heatmap
plt.imshow(superimposed_img)
cv2.imwrite(output_path, superimposed_img)